In [ ]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
import Algo2
import Algo1
import ULA
from sklearn.preprocessing import PolynomialFeatures
from joblib import Parallel, delayed
import ZVnbrosse

In [ ]:
import importlib
importlib.reload(Algo2)

In [ ]:
def f_grad(x):
    a = 1 / np.sqrt(2)
    return x-a+2*a/(1 + np.exp(2* (x * a)))

def f(x):
    a = 1 / np.sqrt(2)
    return 1/2 * (x-a)**2 - np.log(1 + np.exp(-2 * x * a))

Approximation results: use $Q_{l-p}$ to approximate family of $Q_{l,p}$ relying on approximate stationarity of the chain.

In [ ]:
#generate samples from mixture of normals
N_burn = 10000
N_train = 2000
gamma = 0.1
X_train = np.zeros(N_train,dtype = float)

np.random.seed(42)
x0 = np.random.randn()
x_cur = x0
#burn-in
for i in range(N_burn):
    x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
#training sample
for i in range(N_train):
    X_train[i] = x_cur
    x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
X_last = X_train[-1]

Optimize coefficients

In [ ]:
#degree
max_deg = 5
#lag order
lag = 30
#polynomial coefficients
coefs_hermite = np.zeros((lag,max_deg+1),dtype = float)

for i in range(lag):
    y = X_train[i+1:]
    x = X_train[:-(i+1)]
    res = np.polynomial.hermite_e.hermefit(x,y,max_deg)
    coefs_hermite[i,:] = res
#transform into usual polynomials
coefs_poly = np.zeros_like(coefs_hermite)
for i in range(lag):
    coefs_poly[i,:] = np.polynomial.hermite_e.herme2poly(coefs_hermite[i,:])
print(coefs_poly)

In [ ]:
sp.special.factorial2(21, exact=False)
moments_stand_norm = np.zeros(2*max_deg+1,dtype = float)
for i in range(len(moments_stand_norm)):
    moments_stand_norm[i] = sp.special.factorial2(i+1, exact=False)
#eliminate odd
moments_stand_norm[1::2] = 0
print(moments_stand_norm)

In [ ]:
N_burn = 10000
N_test = 1000 #size of test part
N_min = 100 #minimal number of observations to compute \pi_N
gamma = 0.1
X_test = np.zeros(N_test,dtype = float)
Noise = np.zeros_like(X_test)
N_traj = 10

test_stat_vanilla = np.zeros((N_traj,N_test),dtype = float)
test_stat_vr = np.zeros((N_traj,N_test),dtype = float)

In [ ]:
for ind in range(N_traj):
    np.random.seed(1453 + ind)
    #x0 = np.random.randn()
    #x_cur = x0
    #burn-in
    #for i in range(N_burn):
        #x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*np.random.randn()
    #training sample
    x_cur = X_last
    for i in range(N_test):
        Noise[i] = np.random.randn()
        x_cur = x_cur - gamma*f_grad(x_cur) + np.sqrt(2*gamma)*Noise[i]
        X_test[i] = x_cur
    #compute polynomials at Z_l
    poly_vals = np.zeros((max_deg+1,N_test),dtype = float)
    for k in range(max_deg+1):
        c = np.zeros(max_deg+1)
        c[k] = 1
        poly_vals[k,:] = P.hermite_e.hermeval(Noise,c)/np.sqrt((np.sqrt(2*np.pi)*sp.special.factorial(k)))
    f_vals_vanilla = X_test**2
    cvfs = np.zeros_like(f_vals_vanilla)
    for i in range(100,len(cvfs)):
        x = X_test[i]
        #start computing a_{p-l} coefficients
        num_poly = min(lag,i)
        a_vals = np.zeros((num_poly,max_deg+1),dtype = float)
        for npol in range(num_poly):#for a fixed lag Q function
            #compute \hat{a} with fixed lag
            a_cur = np.zeros(max_deg+1,dtype=float)
            for m in range(len(coefs_poly[0])):
                poly_vspom = np.zeros(max_deg+1,dtype=float)
                for u in range(m+1):
                    poly_vspom[u] = ((x-gamma*f_grad(x))**(m-u))*((np.sqrt(2*gamma))**u)*sp.special.binom(m,u)
                a_cur = P.polynomial.polyadd(a_cur,coefs_poly[npol,m]*poly_vspom) 
            for k in range(max_deg+1):
                c = np.zeros(max_deg+1)
                c[k] = 1
                herm_coef = P.hermite_e.herme2poly(c)
                #normalize now
                herm_coef = herm_coef / np.sqrt((np.sqrt(2*np.pi)*sp.special.factorial(k)))
                integr_coefs = P.polynomial.polymul(herm_coef,a_cur)
                a_vals[npol,k] = np.dot(integr_coefs,moments_stand_norm[:len(integr_coefs)])
            #OK, now I have coefficients of the polynomial, and I need to integrate it w.r.t. Gaussian measure
        cvfs[i] = np.sum(a_vals*(poly_vals[:,i-num_poly:i].T))
        #cvfs[i] = np.sum(np.mean(a_vals,axis = 0))
        if (i%100 == 0):
            print("100 observations proceeded")
        #save results
        test_stat_vanilla[ind,i] = np.mean(f_vals_vanilla[:i])
        test_stat_vr[ind,i] = test_stat_vanilla[ind,i] - cvfs[i]/i

In [ ]:
print(test_stat_vanilla[0,:])
print(test_stat_vr[0,:])

In [ ]:
vars_vanilla = np.var(test_stat_vanilla,axis = 0)
vars_adj = np.var(test_stat_vr,axis = 0)
print(vars_vanilla[N_min:N_min+10])
print(vars_adj[N_min:N_min+10])
print(np.mean(vars_adj[N_min:]/vars_vanilla[N_min:]))